In [44]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import lightgbm as lgb

In [45]:
train_data = pd.read_csv('./train_noclean.csv')
test_data = pd.read_csv('./test_noclean.csv')

y_train = train_data.loc[:,'price']
train_data = train_data.drop(columns='price')
num_features = len(train_data.columns)
features = list(train_data.columns[2:num_features])
X_train = train_data.loc[:,features]

features = list(test_data.columns[2:num_features])

X_test = test_data.loc[:,features]
test_labels = test_data.loc[:,'id']

test_labels = test_labels.to_frame()

In [46]:
def run_model(X_train, y_train, learn, max_depth, num_class, num_leaves, min_data_in_leaf, rounds):
    params={}
    params['learning_rate']=0.03
    params['boosting_type']='gbdt' #GradientBoostingDecisionTree
    params['objective']='multiclass' #Multi-class target feature
    params['metric']='multi_error' #metric for multi-class
    params['max_depth']=max_depth
    params['num_class']=num_class
    params['num_leaves']=num_leaves
    clf=lgb.train(params,d_train,rounds)

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

kf = KFold(n_splits = 10, shuffle=True)


X = np.concatenate([X_train], axis= 0)
y = np.concatenate([y_train], axis= 0)
param_grid = {
    'n_estimators': [100, 500, 1000, 1500],
    'colsample_bytree': [0.7, 0.8],
    'max_depth': [10, 15,20,25],
    'num_leaves': [10, 20, 30, 40, 50],
    'reg_lambda': [0.2, 0.5, 0.8, 1, 1.1],
    'reg_alpha' : [0.2, 0.5, 0.8, 1, 1.1], 
    'min_split_gain': [0.3, 0.4],
    'subsample': [0.7, 0.8, 0.9],
    'subsample_freq': [20], 
    'min_data_in_leaf': [15, 25, 30, 40],
    'learning_rate': [0.01, 0.05, 0.1, 0.15]
}
lgb_estimator = lgb.LGBMClassifier(boosting_type='gbdt',  objective='multiclass', metric='multi_error')
rsearch = RandomizedSearchCV(estimator=lgb_estimator, param_distributions=param_grid, cv=kf, n_iter = 5000, n_jobs = -1, verbose=10)
lgb_model = rsearch.fit(X=X, y=y)


print(lgb_model.best_params_, lgb_model.best_score_)


Fitting 10 folds for each of 5000 candidates, totalling 50000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    8.7s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:   10.0s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:   11.1s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:   13.0s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:   15.5s
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:   18.8s
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:   27.2s
[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:   32.1s
[Parallel(n_jobs=-1)]: Done 113 tasks      | elapsed:   45.6s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:   58.2s
[Parallel(n_jobs=-1)]: Done 149 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 189 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 210 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 233 tasks      | elapsed:  

In [48]:
print(lgb_model.best_params_, lgb_model.best_score_)


{'subsample_freq': 20, 'subsample': 0.9, 'reg_lambda': 1.1, 'reg_alpha': 0.8, 'num_leaves': 30, 'n_estimators': 1500, 'min_split_gain': 0.4, 'min_data_in_leaf': 15, 'max_depth': 10, 'learning_rate': 0.01, 'colsample_bytree': 0.7} 0.5615135310322475


In [49]:
print(lgb_model.best_params_, lgb_model.best_score_)


{'subsample_freq': 20, 'subsample': 0.9, 'reg_lambda': 1.1, 'reg_alpha': 0.8, 'num_leaves': 30, 'n_estimators': 1500, 'min_split_gain': 0.4, 'min_data_in_leaf': 15, 'max_depth': 10, 'learning_rate': 0.01, 'colsample_bytree': 0.7} 0.5615135310322475


In [50]:
from sklearn.metrics import accuracy_score
import lightgbm as lgb
kf = KFold(n_splits = 10, shuffle=True)


X = np.concatenate([X_train], axis= 0)
y = np.concatenate([y_train], axis= 0)

for train, test in kf.split(X_train):
    X_train_kf, X_test_kf = X[train],X[test]
    y_train_kf, y_test_kf = y[train],y[test]
    d_train=lgb.Dataset(X_train_kf, label=y_train_kf)
    best_param = {'verbose':-1,'boosting_type':'gbdt',  'objective':'multiclass', 'learning_rate':0.01, 'num_class':5, 'metric':'multi_error','subsample_freq': 20, 'subsample': 0.9, 'reg_lambda': 1.1, 'reg_alpha': 0.8, 'num_leaves': 30, 'n_estimators': 1500, 'min_split_gain': 0.4, 'min_data_in_leaf': 15, 'max_depth': 10, 'learning_rate': 0.01, 'colsample_bytree': 0.7}
    clf=lgb.train(best_param,d_train )
    
    y_pred=clf.predict(X_test_kf)
    y_pred = [np.argmax(line) for line in y_pred]
    print(accuracy_score(y_test_kf, y_pred))

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:151: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


0.5490196078431373
0.5433884297520661
0.5475206611570248
0.5361570247933884
0.5506198347107438
0.5919421487603306
0.5547520661157025
0.5588842975206612
0.5516528925619835
0.5506198347107438


In [7]:
from sklearn.metrics import accuracy_score
import lightgbm as lgb
kf = KFold(n_splits = 10, shuffle=True)


X = np.concatenate([X_train], axis= 0)
y = np.concatenate([y_train], axis= 0)

for train, test in kf.split(X_train):
    X_train_kf, X_test_kf = X[train],X[test]
    y_train_kf, y_test_kf = y[train],y[test]
    d_train=lgb.Dataset(X_train_kf, label=y_train_kf)
    best_param = {'verbose':-1,'boosting_type':'gbdt',  'objective':'multiclass', 'learning_rate':0.01, 'num_class':5, 'metric':'multi_error','subsample_freq': 20, 'subsample': 0.8, 'reg_lambda': 1.2, 'num_leaves': 40, 'n_estimators': 1000, 'min_split_gain': 0.4, 'min_data_in_leaf': 25, 'max_depth': 20, 'colsample_bytree': 0.7}
    clf=lgb.train(best_param,d_train )
    
    y_pred=clf.predict(X_test_kf)
    y_pred = [np.argmax(line) for line in y_pred]
    print(accuracy_score(y_test_kf, y_pred))

0.5490196078431373
0.5413223140495868
0.5712809917355371
0.5609504132231405
0.5423553719008265
0.5599173553719008
0.5547520661157025
0.5568181818181818
0.5599173553719008
0.5712809917355371


In [51]:
X = np.concatenate([X_train], axis= 0)
y = np.concatenate([y_train], axis= 0)
best_param = {'verbose':-1,'boosting_type':'gbdt',  'objective':'multiclass', 'learning_rate':0.01, 'num_class':5, 'metric':'multi_error','subsample_freq': 20, 'subsample': 0.9, 'reg_lambda': 1.1, 'reg_alpha': 0.8, 'num_leaves': 30, 'n_estimators': 1500, 'min_split_gain': 0.4, 'min_data_in_leaf': 15, 'max_depth': 10, 'learning_rate': 0.01, 'colsample_bytree': 0.7}

best_param1 = {'boosting_type':'gbdt',  'objective':'multiclass', 'learning_rate':0.01, 'num_class':5, 'metric':'multi_error','subsample_freq': 20, 'subsample': 0.8, 'reg_lambda': 1.2, 'num_leaves': 40, 'n_estimators': 1000, 'min_split_gain': 0.4, 'min_data_in_leaf': 25, 'max_depth': 20, 'colsample_bytree': 0.7}
d_train=lgb.Dataset(X, y)
clf=lgb.train(best_param,d_train)

In [52]:
y_pred1=clf.predict(X)
y_pred1 = [np.argmax(line) for line in y_pred1]
print(accuracy_score(y, y_pred1))

0.7984712323107117


In [53]:
y_pred=clf.predict(X_test)

In [54]:
print(y_pred)

[[9.51281332e-16 1.63394248e-01 5.43025854e-01 2.24630769e-01
  6.89491292e-02]
 [9.43935377e-16 6.84971007e-02 3.78107808e-01 4.53717870e-01
  9.96772210e-02]
 [9.23656389e-16 2.00696364e-01 4.83803369e-01 2.61079678e-01
  5.44205898e-02]
 ...
 [1.00796018e-15 7.39052999e-02 2.42280576e-01 3.87629532e-01
  2.96184592e-01]
 [3.94127952e-16 9.01571038e-01 5.30528908e-02 3.31832918e-02
  1.21927795e-02]
 [1.33005010e-15 1.43147005e-01 3.97415504e-01 2.89278527e-01
  1.70158964e-01]]


In [55]:
y_pred = [np.argmax(line) for line in y_pred]

In [56]:
print(y_pred)

[2, 3, 2, 2, 2, 4, 4, 1, 3, 2, 1, 1, 1, 3, 1, 1, 1, 2, 2, 4, 3, 2, 2, 4, 2, 4, 4, 4, 1, 4, 3, 3, 1, 1, 3, 4, 4, 1, 1, 1, 4, 4, 4, 1, 4, 4, 4, 1, 2, 2, 2, 3, 1, 1, 1, 3, 1, 4, 2, 2, 2, 4, 1, 3, 1, 4, 1, 4, 2, 3, 4, 3, 2, 3, 3, 2, 4, 3, 1, 1, 2, 3, 1, 3, 3, 4, 1, 3, 1, 1, 2, 4, 3, 2, 1, 4, 4, 2, 3, 2, 1, 1, 2, 4, 1, 1, 1, 3, 1, 1, 3, 1, 4, 3, 3, 2, 4, 4, 3, 1, 1, 4, 2, 1, 2, 3, 3, 1, 1, 4, 3, 2, 3, 3, 1, 4, 3, 4, 4, 2, 4, 3, 4, 4, 4, 3, 4, 4, 2, 4, 1, 1, 4, 2, 3, 2, 3, 3, 4, 1, 3, 2, 3, 4, 4, 2, 2, 4, 1, 1, 4, 1, 1, 4, 1, 2, 2, 4, 4, 3, 2, 4, 4, 2, 3, 3, 1, 3, 2, 3, 3, 2, 1, 3, 1, 1, 3, 1, 1, 1, 4, 3, 1, 1, 2, 3, 2, 3, 4, 3, 2, 1, 2, 2, 4, 3, 2, 3, 3, 4, 2, 4, 2, 1, 1, 3, 3, 4, 2, 4, 3, 3, 3, 3, 3, 4, 2, 2, 4, 1, 2, 1, 2, 2, 4, 2, 1, 2, 4, 4, 2, 1, 1, 1, 4, 1, 2, 2, 2, 4, 3, 2, 3, 1, 3, 1, 1, 1, 2, 2, 3, 4, 1, 1, 3, 3, 1, 1, 3, 3, 2, 2, 3, 4, 2, 2, 3, 3, 2, 4, 2, 3, 1, 2, 3, 1, 3, 2, 1, 4, 4, 3, 4, 1, 2, 3, 1, 1, 3, 3, 4, 2, 4, 1, 4, 4, 4, 1, 3, 4, 1, 2, 2, 2, 1, 3, 1, 1, 3, 2, 3, 3, 3, 

In [57]:
df = pd.DataFrame
test_labels['price'] = y_pred

test_labels['price'] = test_labels['price'].map(lambda x: int(x))

In [58]:
test_labels

,id,price
0,7715,2
1,13196,3
2,13194,2
3,4673,2
4,11325,2
...,...,...
4144,12921,2
4145,7174,1
4146,9240,3
4147,11663,1


In [59]:
test_labels.to_csv(r'./submission8.csv', index=False)

In [ ]:
from sklearn.model_selection import GridSearchCV

kf = KFold(n_splits = 10, shuffle=True)


X = np.concatenate([X_train], axis= 0)
y = np.concatenate([y_train], axis= 0)
param_grid = {
    'n_estimators': [100, 400, 700, 1000],
    'colsample_bytree': [0.7, 0.8],
    'max_depth': [8,10,20],
    'num_leaves': [15, 100, 200],
    'reg_alpha': [1.1, 1.2, 1.3],
    'reg_lambda': [1.1, 1.2, 1.3],
    'min_split_gain': [0.3, 0.4],
    'subsample': [0.7, 0.8, 0.9],
    'subsample_freq': [20]
}
lgb_estimator = lgb.LGBMClassifier(boosting_type='gbdt',  objective='multiclass', learning_rate=0.01, metric='multi_error')
gsearch = GridSearchCV(estimator=lgb_estimator, param_grid=param_grid, cv=kf, verbose=2, n_jobs=-1)
lgb_model = gsearch.fit(X=X, y=y)


print(lgb_model.best_params_, lgb_model.best_score_)
